In [1]:
from sre_parse import Tokenizer
from keras.datasets import reuters
import numpy as np
import pandas as pd

(x_train, y_train), (x_test, y_test) = reuters.load_data(   # 로이터 기사 뉴스
    num_words=10000, test_split=0.2
)

print(x_train)
print(x_train.shape, x_test.shape)    #(8982,) (2246, ) => 8982개와 2246개의 리스트  => 총 11,228개
print(y_train)    
print(np.unique(y_train, return_counts=True))    
print(y_train.shape, y_test.shape)    #(8982,) (2246, ) => 8982개와 2246개의 리스트  => 총 11,228개
print(len(np.unique(y_train)))   # 46개의 label이 있음, 다중분류 

print(type(x_train), type(y_train))   # <class 'numpy.ndarray'> <class 'numpy.ndarray'>
print(type(x_train[0]))               # <class 'list'>
# print(x_train[0].shape)             # AttributeError: 'list' object has no attribute 'shape'
print(len(x_train[0]))                # 87
print(len(x_train[1]))                # 56


#전처리
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

x_train = pad_sequences(x_train, padding='pre', maxlen=100, truncating='pre')
                        #shape=(8982,) => (8982, 100)
x_test = pad_sequences(x_test, padding='pre', maxlen=100, truncating='pre')

# reshape => LSTM과 Conv1D는 reshape으로 3차원으로 만들어 주어야 함
x_train = x_train.reshape(8982, 100, 1)
x_test = x_test.reshape(2246, 100, 1)

print(x_train.shape, y_train.shape)     # (8982, 100, 1) (8982, 46)
print(x_test.shape, y_test.shape)       # (2246, 100, 1) (2246, 46)

[list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])
 list([1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12])
 list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110, 59, 89, 90, 67, 139

/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
y_train = to_categorical(y_train)                        
y_test = to_categorical(y_test)                        

print(x_train.shape, y_train.shape)     # (8982, 100) (8982, 46)
print(x_test.shape, y_test.shape)       # (2246, 100) (2246, 46)

# y_train = y_train.reshape(8982, 46, 1)
# y_test = y_test.reshape(2246, 46, 1)

# print(x_train.shape, y_train.shape)     # (8982, 100) (8982, 46)
# print(x_test.shape, y_test.shape) 

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten

model = Sequential()
# model.add(Embedding(10000, 20, input_length=100))    
model.add(LSTM(64, input_shape=(100,1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(46, activation='softmax'))

model.summary()

In [ ]:
# 3. 컴파일, 훈련
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=10, batch_size=16)


#4. 평가, 예측
loss, acc = model.evaluate(x_test, y_test)  # [0]이면 loss, [1]이면 accuracy 
print('loss : ', loss, 'acc : ', acc)

Epoch 1/10
562/562 [==============================] - 79s 141ms/step - loss: 2.2894 - acc: 0.4210
Epoch 2/10
562/562 [==============================] - 72s 129ms/step - loss: 2.1214 - acc: 0.4667
Epoch 3/10
562/562 [==============================] - 75s 133ms/step - loss: 2.0841 - acc: 0.4663
Epoch 4/10
261/562 [============>.................] - ETA: 38s - loss: 2.0461 - acc: 0.4672